In [1]:
"""
File v721_optima_increase.py

Michel Bierlaire
Mon Aug 31 14:35:46 2020

"""
import sys
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models
import biogeme.results as res
from biogeme.expressions import Beta

# Read the data
df = pd.read_csv('optima.dat', '\t')
database = db.Database('boeing', df)

# The following statement allows you to use the names of the
# variable as Python variable.
globals().update(database.variables)

# Define new variables
TimePT_scaled = TimePT / 200
TimeCar_scaled = TimeCar / 200
MarginalCostPT_scaled = MarginalCostPT / 10
CostCarCHF_scaled = CostCarCHF / 10
distance_km_scaled = distance_km / 5
male = Gender == 1
female = Gender == 2
unreportedGender = Gender == -1
fulltime = OccupStat == 1
notfulltime = OccupStat != 1

# Removing some observations
exclude = ((Choice == -1) + ((CarAvail == 3) * (Choice == 1))) != 0
database.remove(exclude)

# List of parameters to be estimated
ASC_CAR = Beta('ASC_CAR', 0, None, None, 0)
ASC_SM = Beta('ASC_SM', 0, None, None, 0)
BETA_TIME_FULLTIME = Beta('BETA_TIME_FULLTIME', 0, None, None, 0)
BETA_TIME_OTHER = Beta('BETA_TIME_OTHER', 0, None, None, 0)
BETA_DIST_MALE = Beta('BETA_DIST_MALE', 0, None, None, 0)
BETA_DIST_FEMALE = Beta('BETA_DIST_FEMALE', 0, None, None, 0)
BETA_DIST_UNREPORTED = Beta('BETA_DIST_UNREPORTED', 0, None, None, 0)
BETA_COST = Beta('BETA_COST', 0, None, None, 0)

# We define a function that takes the increase of gas cost as input
# and returns the market shares

def calculateMarketShares(increase):
    # Definition of utility functions:
    V_PT = BETA_TIME_FULLTIME * TimePT_scaled * fulltime + \
        BETA_TIME_OTHER * TimePT_scaled * notfulltime + \
        BETA_COST * MarginalCostPT_scaled
    V_CAR = ASC_CAR + \
        BETA_TIME_FULLTIME * TimeCar_scaled * fulltime + \
        BETA_TIME_OTHER * TimeCar_scaled * notfulltime + \
        BETA_COST * CostCarCHF_scaled * (1 + increase)
    V_SM = ASC_SM + \
        BETA_DIST_MALE * distance_km_scaled * male + \
        BETA_DIST_FEMALE * distance_km_scaled * female + \
        BETA_DIST_UNREPORTED * distance_km_scaled * unreportedGender

    # Associate utility functions with the numbering of alternatives
    V = {0: V_PT,
         1: V_CAR,
         2: V_SM}

    try:
        m = 'v715_optima_model'
        results = res.bioResults(pickleFile=f'{m}.pickle')
    except FileNotFoundError:
        print(f'Run first the script {m}.py in order to generate the file '
              '{m}.pickle.')
        sys.exit()

    prob_pt = models.logit(V, None, 0)
    prob_car = models.logit(V, None, 1)
    prob_sm = models.logit(V, None, 2)

    simulate = {'Prob. public transp.': prob_pt,
                'Prob. car': prob_car,
                'Prob. slow modes': prob_sm,
                'Weight': Weight}

    biosim = bio.BIOGEME(database, simulate)
    biosim.modelName = 'v715_optima_simul'

    # Perform the simulation
    simresults = biosim.simulate(results.getBetaValues())

    totalWeight = simresults['Weight'].sum()

    simresults['Weighted public'] = simresults['Prob. public transp.'] * simresults['Weight']
    simresults['Weighted car'] = simresults['Prob. car'] * simresults['Weight']
    simresults['Weighted slow'] = simresults['Prob. slow modes'] * simresults['Weight']


    pt = simresults['Weighted public'].sum() / totalWeight
    car = simresults['Weighted car'].sum() / totalWeight
    sm = simresults['Weighted slow'].sum() / totalWeight
    return pt, car, sm

for i in [0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3]:
    ms_pt, ms_car, ms_sm = calculateMarketShares(i)
    print(f'Market shares with increase {100*i:2.0f}%:\t'
          f'PT [{100*ms_pt:.2f}%], '
          f'car [{100*ms_car:.2f}%], '
          f'slow modes [{100*ms_sm:.2f}%]')

Market shares with increase  0%:	PT [28.70%], car [65.22%], slow modes [6.08%]
Market shares with increase  5%:	PT [29.00%], car [64.91%], slow modes [6.10%]
Market shares with increase 10%:	PT [29.30%], car [64.59%], slow modes [6.11%]
Market shares with increase 15%:	PT [29.60%], car [64.28%], slow modes [6.12%]
Market shares with increase 20%:	PT [29.90%], car [63.97%], slow modes [6.14%]
Market shares with increase 25%:	PT [30.20%], car [63.65%], slow modes [6.15%]
Market shares with increase 30%:	PT [30.50%], car [63.33%], slow modes [6.17%]
